In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_exp")

2023/05/24 10:20:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/05/24 10:20:12 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/mnt/e/projects/mlops-zoomcamp/mlflow/notebooks/hw/mlruns/1', creation_time=1684948819406, experiment_id='1', last_update_time=1684948819406, lifecycle_stage='active', name='nyc_taxi_exp', tags={}>

# Preprocessing

In [3]:
pd.read_parquet("../data/yellow_tripdata_2022-01.parquet")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [13]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.loc[:, 'duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe("./data/yellow_tripdata_2022-01.parquet")
df_val = read_dataframe("./data/yellow_tripdata_2022-02.parquet")

In [16]:
df_train.shape

(2421440, 20)

In [20]:
(2463930 - 2421440)/2463931

0.017244801092238377

In [19]:
100-0.017245206947759495

99.98275479305224

In [5]:
df_train.loc[:, 'PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val.loc[:, 'PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO'] #['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
lr = LinearRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.75871520559622

In [9]:
with open("models/lin_reg.bin", 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [11]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Amirhossein")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    lr = Ridge(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

## Trying XGBoost

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, STATUS_OK, Trials, hp
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_chiled_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[17:37:11] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.  
[17:37:11] WARNING: ../src/learner.cc:767:                                                                              
Parameters: { "min_chiled_weight" } are not used.

[0]	validation-rmse:19.53919                                                                                            
[1]	validation-rmse:18.05441                                                                                            
[2]	validation-rmse:16.72469                                                                                            
[3]	validation-rmse:15.53334                                                                                            
[4]	validation-rmse:14.47018                                                                                            
[5]	validation-rmse:13.52416                                                                          

[65]	validation-rmse:6.59236                                                                                            
[66]	validation-rmse:6.59110                                                                                            
[67]	validation-rmse:6.58998                                                                                            
[68]	validation-rmse:6.58851                                                                                            
[69]	validation-rmse:6.58730                                                                                            
[70]	validation-rmse:6.58594                                                                                            
[71]	validation-rmse:6.58502                                                                                            
[72]	validation-rmse:6.58405                                                                                            
[73]	validation-rmse:6.58321    

[132]	validation-rmse:6.54935                                                                                           
[133]	validation-rmse:6.54856                                                                                           
[134]	validation-rmse:6.54816                                                                                           
[135]	validation-rmse:6.54789                                                                                           
[136]	validation-rmse:6.54724                                                                                           
[137]	validation-rmse:6.54643                                                                                           
[138]	validation-rmse:6.54613                                                                                           
[139]	validation-rmse:6.54557                                                                                           
[140]	validation-rmse:6.54503   

[199]	validation-rmse:6.52445                                                                                           
[200]	validation-rmse:6.52412                                                                                           
[201]	validation-rmse:6.52375                                                                                           
[202]	validation-rmse:6.52349                                                                                           
[203]	validation-rmse:6.52289                                                                                           
[204]	validation-rmse:6.52267                                                                                           
[205]	validation-rmse:6.52236                                                                                           
[206]	validation-rmse:6.52167                                                                                           
[207]	validation-rmse:6.52128   

[266]	validation-rmse:6.50425                                                                                           
[267]	validation-rmse:6.50412                                                                                           
[268]	validation-rmse:6.50373                                                                                           
[269]	validation-rmse:6.50346                                                                                           
[270]	validation-rmse:6.50302                                                                                           
[271]	validation-rmse:6.50283                                                                                           
[272]	validation-rmse:6.50273                                                                                           
[273]	validation-rmse:6.50254                                                                                           
[274]	validation-rmse:6.50212   

...

[732]	validation-rmse:6.53363                                                                                           
[733]	validation-rmse:6.53360                                                                                           
[734]	validation-rmse:6.53351                                                                                           
[735]	validation-rmse:6.53343                                                                                           
[736]	validation-rmse:6.53312                                                                                           
[737]	validation-rmse:6.53294                                                                                           
[738]	validation-rmse:6.53283                                                                                           
[739]	validation-rmse:6.53280                                                                                           
[740]	validation-rmse:6.53290   

[799]	validation-rmse:6.52539                                                                                           
[800]	validation-rmse:6.52535                                                                                           
[801]	validation-rmse:6.52531                                                                                           
[802]	validation-rmse:6.52530                                                                                           
[803]	validation-rmse:6.52525                                                                                           
[804]	validation-rmse:6.52519                                                                                           
[805]	validation-rmse:6.52510                                                                                           
[806]	validation-rmse:6.52501                                                                                           
[807]	validation-rmse:6.52481   

[866]	validation-rmse:6.51918                                                                                           
[867]	validation-rmse:6.51878                                                                                           
[868]	validation-rmse:6.51877                                                                                           
[869]	validation-rmse:6.51879                                                                                           
[870]	validation-rmse:6.51853                                                                                           
[871]	validation-rmse:6.51823                                                                                           
[872]	validation-rmse:6.51824                                                                                           
[873]	validation-rmse:6.51806                                                                                           
[874]	validation-rmse:6.51793   

[933]	validation-rmse:6.51016                                                                                           
[934]	validation-rmse:6.51027                                                                                           
[935]	validation-rmse:6.51007                                                                                           
[936]	validation-rmse:6.50993                                                                                           
[937]	validation-rmse:6.50990                                                                                           
[938]	validation-rmse:6.51003                                                                                           
[939]	validation-rmse:6.51017                                                                                           
[940]	validation-rmse:6.50994                                                                                           
[941]	validation-rmse:6.50976   

KeyboardInterrupt: 

In [28]:
params = {
    'learning_rate': 0.15092373074767682,
    'max_depth': 14,
    'min_chiled_weight': 1.2852486038273228,
    'objective': 'reg:linear',
    'reg_alpha': 0.3153264660136246,
    'reg_lambda': 0.07879954689330733,
    'seed': 42,
}

mlflow.xgboost.autolog()
    
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2023/05/06 17:57:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2acc5371cacb490f8a7c792464b224c2', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[17:57:01] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[17:57:01] WARNING: ../src/learner.cc:767: 
Parameters: { "min_chiled_weight" } are not used.

[0]	validation-rmse:18.52140
[1]	validation-rmse:16.29441
[2]	validation-rmse:14.44843
[3]	validation-rmse:12.92629
[4]	validation-rmse:11.68142
[5]	validation-rmse:10.66813
[6]	validation-rmse:9.84744
[7]	validation-rmse:9.19133
[8]	validation-rmse:8.67027
[9]	validation-rmse:8.25188
[10]	validation-rmse:7.92276
[11]	validation-rmse:7.66331
[12]	validation-rmse:7.45681
[13]	validation-rmse:7.29506
[14]	validation-rmse:7.16581
[15]	validation-rmse:7.06452
[16]	validation-rmse:6.98266
[17]	validation-rmse:6.91693
[18]	validation-rmse:6.86488
[19]	validation-rmse:6.82037
[20]	validation-rmse:6.78654
[21]	validation-rmse:6.75779
[22]	validation-rmse:6.73345
[23]	validation-rmse:6.71336
[24]	validation-rmse:6.69778
[25]	validation-rmse:6.68347
[26]	validation-rmse:6.67197
[27]	va

[270]	validation-rmse:6.48877
[271]	validation-rmse:6.48839
[272]	validation-rmse:6.48791
[273]	validation-rmse:6.48730
[274]	validation-rmse:6.48702
[275]	validation-rmse:6.48673
[276]	validation-rmse:6.48647
[277]	validation-rmse:6.48602
[278]	validation-rmse:6.48560
[279]	validation-rmse:6.48536
[280]	validation-rmse:6.48491
[281]	validation-rmse:6.48480
[282]	validation-rmse:6.48469
[283]	validation-rmse:6.48434
[284]	validation-rmse:6.48397
[285]	validation-rmse:6.48403
[286]	validation-rmse:6.48408
[287]	validation-rmse:6.48377
[288]	validation-rmse:6.48333
[289]	validation-rmse:6.48304
[290]	validation-rmse:6.48284
[291]	validation-rmse:6.48253
[292]	validation-rmse:6.48229
[293]	validation-rmse:6.48217
[294]	validation-rmse:6.48160
[295]	validation-rmse:6.48117
[296]	validation-rmse:6.48133
[297]	validation-rmse:6.48110
[298]	validation-rmse:6.48075
[299]	validation-rmse:6.48077
[300]	validation-rmse:6.48064
[301]	validation-rmse:6.47977
[302]	validation-rmse:6.47931
[303]	vali

[544]	validation-rmse:6.43283
[545]	validation-rmse:6.43286
[546]	validation-rmse:6.43284
[547]	validation-rmse:6.43261
[548]	validation-rmse:6.43233
[549]	validation-rmse:6.43234
[550]	validation-rmse:6.43223
[551]	validation-rmse:6.43229
[552]	validation-rmse:6.43198
[553]	validation-rmse:6.43192
[554]	validation-rmse:6.43146
[555]	validation-rmse:6.43104
[556]	validation-rmse:6.43099
[557]	validation-rmse:6.43096
[558]	validation-rmse:6.43060
[559]	validation-rmse:6.43060
[560]	validation-rmse:6.43057
[561]	validation-rmse:6.43035
[562]	validation-rmse:6.43062
[563]	validation-rmse:6.43060
[564]	validation-rmse:6.43038
[565]	validation-rmse:6.43019
[566]	validation-rmse:6.42980
[567]	validation-rmse:6.42978
[568]	validation-rmse:6.42957
[569]	validation-rmse:6.42914
[570]	validation-rmse:6.42919
[571]	validation-rmse:6.42891
[572]	validation-rmse:6.42907
[573]	validation-rmse:6.42872
[574]	validation-rmse:6.42832
[575]	validation-rmse:6.42813
[576]	validation-rmse:6.42803
[577]	vali

[818]	validation-rmse:6.39967
[819]	validation-rmse:6.39970
[820]	validation-rmse:6.39968
[821]	validation-rmse:6.39947
[822]	validation-rmse:6.39923
[823]	validation-rmse:6.39900
[824]	validation-rmse:6.39902
[825]	validation-rmse:6.39927
[826]	validation-rmse:6.39926
[827]	validation-rmse:6.39879
[828]	validation-rmse:6.39873
[829]	validation-rmse:6.39854
[830]	validation-rmse:6.39837
[831]	validation-rmse:6.39809
[832]	validation-rmse:6.39797
[833]	validation-rmse:6.39770
[834]	validation-rmse:6.39752
[835]	validation-rmse:6.39724
[836]	validation-rmse:6.39720
[837]	validation-rmse:6.39721
[838]	validation-rmse:6.39715
[839]	validation-rmse:6.39718
[840]	validation-rmse:6.39736
[841]	validation-rmse:6.39732
[842]	validation-rmse:6.39739
[843]	validation-rmse:6.39746
[844]	validation-rmse:6.39748
[845]	validation-rmse:6.39740
[846]	validation-rmse:6.39734
[847]	validation-rmse:6.39694
[848]	validation-rmse:6.39689
[849]	validation-rmse:6.39668
[850]	validation-rmse:6.39681
[851]	vali

2023/05/06 18:02:11 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/06 18:02:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/amir/miniconda3/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
